# 02. Agent Workflow Demo

This notebook demonstrates the complete workflow of AgentCore, from planning to execution.

## Workflow Steps
1. Task Planning
2. Plan Validation
3. Execution
4. Result Processing
5. Memory Storage

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append('..')

from agents.planning_agent import PlanningAgent
from agents.execution_agent import ExecutionAgent
from utils.logging_panel import BaseLoggingPanel
from llm.interface import OllamaLLMProvider
from utils.memory_interface import VectorMemoryProvider, ErrorMemoryProvider
from utils.path_utils import ensure_directory_exists, get_project_root

# Get project root and ensure data directories exist
project_root = get_project_root()
vector_store_dir = ensure_directory_exists("data/vector_store", project_root)
error_store_dir = ensure_directory_exists("data/error_store", project_root)

# Set up shared providers
llm_provider = OllamaLLMProvider(
    model="mistral",
    temperature=0.7,
    max_tokens=500,  # Reduced from 1000 to a more reasonable value
    context_window=8192
)

# Set up shared logger
logger = BaseLoggingPanel()

# Set up memory providers
vector_memory = VectorMemoryProvider(
    config={
        "type": "vector",
        "embedding_model": "mxbai-embed-large",
        "max_history": 10,
        "index_name": "pasta_recipes",
        "base_url": "http://localhost:11434",  # Ollama's default URL
        "persist_directory": vector_store_dir
    }
)

error_memory = ErrorMemoryProvider(
    config={
        "use_vector_store": True,
        "embedding_model": "mxbai-embed-large",
        "max_history": 10,
        "index_name": "error_patterns",
        "base_url": "http://localhost:11434",  # Ollama's default URL
        "persist_directory": error_store_dir
    }
)

# Define the task
task = "Find a suitable homemade pasta recipe that supports hand-twisted pasta shapes.  Provide photos of the pasta shapes. Also create a chart of the interest in time from 1984 onwards."

In [ ]:
# Create planning agent with config
planner = PlanningAgent(
    config={
        "model_type": "ollama",
        "model_name": "mistral",
        "temperature": 0.7,
        "max_tokens": 500,
        "debug": True  # Enable debug mode for streaming output
    },
    llm_provider=llm_provider,
    logger=logger
)

# Generate plan with streaming output
print("\nGenerating plan...")
plan = await planner.plan(task)

# Display the steps
print("\nGenerated Plan:")
for i, step in enumerate(plan, 1):
    print(f"\nStep {i}:")
    print(f"Description: {step['description']}")

In [ ]:
# Create execution agent with separate debug panel
execution_logger = BaseLoggingPanel(title="Execution Agent Debug Logs")
executor = ExecutionAgent(
    config={
        "model_type": "ollama",
        "model_name": "mistral",
        "temperature": 0.7,
        "max_tokens": 500,
        "debug": True
    },
    llm_provider=llm_provider,
    logger=execution_logger  # Use the separate debug panel
)

# Execute each step
results = []
for step in plan:
    print(f"\nExecuting Step {step['step']}:")
    print(f"Description: {step['description']}")
    
    # Execute the step
    result = await executor.execute(step)
    results.append(result)
    
    # Display the result
    print(f"Result: {'Success' if result['success'] else 'Failed'}")
    print(f"Output: {result['output']}")
    if not result['success']:
        print(f"Error: {result['error']}")
        break